In [11]:
import math
import csv
import folium

from geopy.distance import geodesic

In [12]:
def get_hexagon_row_coord(start_coord, radius, count, coord_data):
    if count== 0:
        return coord_data
    else:
        next_coord = geodesic(meters=radius*2).destination(start_coord, 90)
        row = [next_coord.latitude, next_coord.longitude]
        coord_data.append(row)
        return get_hexagon_row_coord(next_coord, radius, count-1, coord_data)

In [13]:
def row_mapper(start_coord, radius, row_count, height_count, coord_data):
    height = radius * math.sqrt(3)
    if height_count == 0:
        return coord_data
    elif height_count % 2 != 0:
        next_coord = geodesic(meters=radius).destination(
            geodesic(meters=height).destination(start_coord, 180), 90
        )
        coord_data = get_hexagon_row_coord(next_coord, radius, row_count, coord_data)
    else:
        next_coord = geodesic(meters=radius).destination(
            geodesic(meters=height).destination(start_coord, 180), 270
        )
        coord_data = get_hexagon_row_coord(next_coord, radius, row_count, coord_data)
    return row_mapper(next_coord, radius, row_count, height_count-1, coord_data)

In [14]:
def data_to_csv(data):
    file_path = 'coordinates/hexagonal-packing-v3.csv'
    with open(file_path, mode='w', newline='', encoding='utf-8') as file:
        writer = csv.writer(file)
        writer.writerow(['lat', 'lon'])
        writer.writerows(data)
        print(f"Coordinates saved to {file_path}")

In [15]:
r = 500
start_point = (-8.096568, 114.437477)
row_count = 140
height_count = 100
coordinates = row_mapper(start_point, r, row_count, height_count, [])

In [16]:
print(type(coordinates))

<class 'list'>


In [17]:
data_to_csv(coordinates)

Coordinates saved to coordinates/hexagonal-packing-v3.csv


In [18]:
world_map = folium.Map(location=start_point, zoom_start=15)

for coord in coordinates:
    folium.Marker(
        location=coord,
        icon=folium.Icon(color='blue', icon='info-sign')
    ).add_to(world_map)

In [19]:
world_map.save('test-map.html')